In [68]:
#Import neo4j Database Driver

from neo4j import GraphDatabase
import time

In [69]:
# Get Connection to neo4j

uri = 'bolt://localhost:7687'
username = 'neo4j'
password = 'graph'

neo4j_driver = GraphDatabase.driver(uri, auth=(username,password))

In [70]:
#Create the YT Graph

Create_Query = "LOAD CSV WITH HEADERS FROM 'file:///root/YT_Data/user_relations1.csv' AS line MERGE (n:YT_User {Name:line.user1}) MERGE (m:YT_User {Name:line.user2}) MERGE (n) -[:uploads_similar_to]-> (m)"

start = time.time()
with neo4j_driver.session() as neo4j_Session:
    neo4j_Session.run(Create_Query)
print()

In [60]:
#Delete self loops and nodes without any edges

Delete_loops_Query = "MATCH (n:YT_User)-[r:uploads_similar_to]->(n) DELETE r"
Delete_leafroot_nodes_Query = "MATCH (n:YT_User) WHERE NOT((n)-->(:YT_User)) AND NOT((n)<--(:YT_User)) DELETE n"

with neo4j_driver.session() as neo4j_Session:
    neo4j_Session.run(Delete_loops_Query)
    neo4j_Session.run(Delete_leafroot_nodes_Query)

In [ ]:
#Run label Propogation

LP_Query = "CALL algo.labelPropagation.stream('YT_User', 'uploads_similar_to', { iterations: 100 }) YIELD nodeId, label RETURN label, collect(algo.getNodeById(nodeId).Name) AS Users ORDER BY size(Users) DESC"

with neo4j_driver.session() as neo4j_Session:
    result = neo4j_Session.run(LP_Query)

In [ ]:
groups = result.data()

In [ ]:
print(groups)